In [72]:
%matplotlib inline
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
plt.rcParams['font.family']='serif' 
plt.rcParams['font.weight']='light'
plt.rcParams['font.size']=14
figsize = (15,5)

In [81]:
glob = pd.read_json('Constants/constants.json')
v = glob['physics']['ckm']['us']
m_mes = glob['physics']['mass']['k+']
m_lep = glob['physics']['mass']['mu']
f_mes = glob['physics']['decayConst']['k+']
g_f = glob['physics']['constants']['Gf']
detW = glob['experiment']['detector']['width']
detH = glob['experiment']['detector']['height']
detL = glob['experiment']['detector']['length']
detD = glob['experiment']['detector']['distance']
pipeL = glob['experiment']['decayPipe']['length']
detD = detD - detL/2.
area = detH*detW
radius = np.sqrt(area/(2*np.pi))

In [82]:
def MesonFlux(flux_mes,p_mes,l):
    '''
    Meson flux as a function of position in the decay pipe.
    '''
    lambda_mes = 3.7*p_mes/m_mes
    flux_mes = flux_mes * np.exp(-l/float(lambda_mes))
    return flux_mes

def ThetaMax(l):
    '''
    Maximum angle with respect to the beam line that a neutrino
    can have in order to go through the detector.
    '''
    thetaMax = np.arccos( (targDetDistance-l) / np.sqrt((targDetDistance-x)**2 + (radius)**2) )
    thetaMax = np.degrees(thetaMax)
    return thetaMax

def ProjectionFunction(x,theta):
    '''
    Projection function. Returns 1 if neutrino trajectory is inside
    the detector. Return 0 otherwise.
    '''
    proj = np.where(abs(theta) >= ThetaMax(x), 0, 1)
    return proj

def LIPS(e_nu,p_mes,theta):
    '''
    Lorentz invariant phase space factor.
    '''
    psFactor = m_mes**2. - m_lep**2. - 2*e_nu*np.sqrt(p_mes**2. + m_mes**2.) + 2.*p_mes*e_nu*np.cos(theta)
    if psFactor == 0:
        return 1
    else:
        return 0

def MatrixElement2():
    '''
    Matrix element for meson 2-body decay.
    '''
    matrixElement2 = 2*(g_f**2.)*(f_mes**2.)*(m_mes**4)*(v**2)*((m_lep/float(m_mes))**2 - (m_lep/float(m_mes))**4)
    return matrixElement2

def DiffDecayWidth(e_mes,e_nu,p_mes,theta):
    '''
    Differential decay width.
    '''
    diffDecayWidth = MatrixElement2()/float(2.*e_mes) * e_nu/float(8.*np.pi**2) * LIPS(e_nu,p_mes,theta)
    return diffDecayWidth

def SourceTerm(flux_mes,p_mes,e_mes,e_nu,theta,l):
    '''
    Source term for the neutrino flux.
    '''
    source = MesonFlux(flux_mes,p_mes,l)*(p_mes/float(m_mes))*DiffDecayWidth(e_mes,e_nu,p_mes,theta)
    return source
    
